In [1]:
#importing libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import math

In [2]:
#load the NSL_KDD dataset
train=pd.read_csv('dataset/KDDTrain+.csv')
test=pd.read_csv('dataset/KDDTest+.csv')

#size of train and test data
print(train.shape,test.shape)

# concatenate traina and test data
dataset=pd.concat([test,train])

#size of whole dataset
print(dataset.shape)

#print(dataset.head(2))
#print(dataset.columns)
X = dataset.iloc[:, :-1].values
print(X)
y = dataset.iloc[:, 41:42].values
print(y)

(113270, 42) (15451, 42)
(128721, 42)
[[0 'tcp' 'private' ... 0.0 1.0 1.0]
 [0 'tcp' 'private' ... 0.0 1.0 1.0]
 [2 'tcp' 'ftp_data' ... 0.0 0.0 0.0]
 ...
 [0 'tcp' 'smtp' ... 0.0 0.01 0.0]
 [0 'tcp' 'klogin' ... 1.0 0.0 0.0]
 [0 'tcp' 'ftp_data' ... 0.0 0.0 0.0]]
[['DDoS']
 ['DDoS']
 ['normal']
 ...
 ['normal']
 ['DDoS']
 ['normal']]


In [3]:
#Encoding of dataset
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1, 2, 3])], remainder = 'passthrough')
X= ct.fit_transform(X)
print(X)

[[0.0 1.0 0.0 ... 0.0 1.0 1.0]
 [0.0 1.0 0.0 ... 0.0 1.0 1.0]
 [0.0 1.0 0.0 ... 0.0 0.0 0.0]
 ...
 [0.0 1.0 0.0 ... 0.0 0.01 0.0]
 [0.0 1.0 0.0 ... 1.0 0.0 0.0]
 [0.0 1.0 0.0 ... 0.0 0.0 0.0]]


In [4]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
y[:, 0] = le_y.fit_transform(y[:, 0])
y= y.astype(int)
y = y.ravel()
print(y)

[0 0 1 ... 1 0 1]


In [5]:
# min_max normalisation between (a,b)
from sklearn.preprocessing import minmax_scale
X = minmax_scale(X, feature_range=(0,1), axis=0)
print(X)

[[0.   1.   0.   ... 0.   1.   1.  ]
 [0.   1.   0.   ... 0.   1.   1.  ]
 [0.   1.   0.   ... 0.   0.   0.  ]
 ...
 [0.   1.   0.   ... 0.   0.01 0.  ]
 [0.   1.   0.   ... 1.   0.   0.  ]
 [0.   1.   0.   ... 0.   0.   0.  ]]


In [6]:
# information gain

from sklearn.feature_selection import mutual_info_classif
# determine the mutual information or information gain
mutual_info = mutual_info_classif(X, y)
mutual_info
mutual_info = pd.Series(mutual_info)
mutual_info.sort_values(ascending=False)

79     0.570033
104    0.478160
103    0.464442
80     0.439379
76     0.418130
97     0.382104
108    0.368668
109    0.363167
107    0.345305
113    0.343953
112    0.339801
99     0.333293
72     0.332544
100    0.332257
86     0.285735
111    0.175151
106    0.173685
25     0.173028
110    0.143554
105    0.110893
47     0.110723
98     0.107711
115    0.057154
14     0.045065
114    0.043446
2      0.040284
78     0.035685
101    0.034368
102    0.032562
69     0.025669
         ...   
51     0.001442
35     0.001428
68     0.001219
16     0.001199
49     0.001104
42     0.001076
81     0.000736
39     0.000730
3      0.000570
77     0.000251
60     0.000154
6      0.000092
50     0.000004
62     0.000000
74     0.000000
83     0.000000
95     0.000000
4      0.000000
85     0.000000
75     0.000000
53     0.000000
73     0.000000
21     0.000000
88     0.000000
94     0.000000
48     0.000000
45     0.000000
59     0.000000
20     0.000000
89     0.000000
Length: 116, dtype: floa

In [7]:
from sklearn.feature_selection import SelectKBest
#No we Will select the  top important features
selector1 = SelectKBest(mutual_info_classif, k=4)
selector1.fit(X, y)
#Get columns to keep and create new dataframe with those only

SelectKBest(k=4,
            score_func=<function mutual_info_classif at 0x0000019BCD0E1378>)

In [8]:
cols = selector1.get_support(indices=True)
XIG = X[:,cols]
print(XIG)

[[0.00000000e+00 0.00000000e+00 4.00000000e-02 6.00000000e-02]
 [0.00000000e+00 0.00000000e+00 1.00000000e-02 6.00000000e-02]
 [1.44929445e-04 0.00000000e+00 1.00000000e+00 0.00000000e+00]
 ...
 [2.49046902e-05 5.46335201e-05 1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 6.00000000e-02 5.00000000e-02]
 [1.68561552e-06 0.00000000e+00 1.00000000e+00 0.00000000e+00]]


In [9]:
print(XIG.shape)

(128721, 4)


In [10]:
# splitting of dataset in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(XIG, y, test_size = 0.2, random_state =42)
print(X_train)
print(X_test)
print(y_train)
print(y_test)

[[3.66146946e-06 7.66576578e-04 1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 8.60000000e-01 2.90000000e-01]
 [1.15202332e-05 0.00000000e+00 1.00000000e+00 0.00000000e+00]
 ...
 [3.21494880e-06 2.51684107e-04 1.00000000e+00 0.00000000e+00]
 [2.41121160e-06 2.78289493e-04 1.00000000e+00 0.00000000e+00]
 [3.24843785e-06 1.54955744e-02 1.00000000e+00 0.00000000e+00]]
[[0.00000000e+00 0.00000000e+00 4.40000000e-01 9.00000000e-02]
 [3.80658868e-06 1.85597466e-03 1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 6.00000000e-02 6.00000000e-02]
 ...
 [2.54516780e-06 1.96339213e-04 1.00000000e+00 0.00000000e+00]
 [5.20419837e-05 3.81751721e-03 1.00000000e+00 0.00000000e+00]
 [2.38888556e-06 9.18383781e-04 1.00000000e+00 0.00000000e+00]]
[1 0 0 ... 1 1 1]
[0 1 0 ... 1 1 1]


In [11]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [12]:
# base classifier RandomForest
y_train = y_train.ravel()
clf1 = RandomForestClassifier(n_estimators=100)
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred1)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred1))
print("RMSE= ",RMSE)

population: 25745
P: 15363
N: 10382
PositiveTest: 15604
NegativeTest: 10141
TP: 15338
TN: 10116
FP: 266
FN: 25
TPR: 0.9983727136626961
TNR: 0.9743787324214987
PPV: 0.9829530889515509
NPV: 0.9975347598856129
FPR: 0.025621267578501253
FDR: 0.017046911048449114
FNR: 0.001627286337303912
ACC: 0.9886968343367645
F1_score: 0.9906028998611425
MCC: 0.9766119868321975
informedness: 0.9727514460841948
markedness: 0.9804878488371638
prevalence: 0.5967372305302
LRP: 38.966562079872595
LRN: 0.0016700757961535403
DOR: 23332.21172932331
FOR: 0.0024652401143871413
RMSE=  0.10631634711198264


In [13]:
# base classifier GradientBoosting
clf2 = GradientBoostingClassifier(n_estimators=100)
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred2)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred2))
print("RMSE= ",RMSE)


population: 25745
P: 15363
N: 10382
PositiveTest: 15611
NegativeTest: 10134
TP: 15333
TN: 10104
FP: 278
FN: 30
TPR: 0.9980472563952353
TNR: 0.973222885763822
PPV: 0.9821920440714881
NPV: 0.9970396684428656
FPR: 0.026777114236178002
FDR: 0.017807955928511948
FNR: 0.0019527436047646943
ACC: 0.9880365119440668
F1_score: 0.9900561761477368
MCC: 0.9752428029062677
informedness: 0.9712701421590573
markedness: 0.9792317125143537
prevalence: 0.5967372305302
LRP: 37.272397898904075
LRN: 0.0020064711109132084
DOR: 18576.094964028776
FOR: 0.002960331557134399
RMSE=  0.10937773107874012


In [14]:
# svm
from sklearn.svm import SVC  
clf3= SVC(kernel='linear') 
clf3.fit(X_train, y_train) 
y_pred3 = clf3.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred3)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred3))
print("RMSE= ",RMSE)

population: 25745
P: 15363
N: 10382
PositiveTest: 16598
NegativeTest: 9147
TP: 15169
TN: 8953
FP: 1429
FN: 194
TPR: 0.9873722580225216
TNR: 0.8623579271816606
PPV: 0.913905289793951
NPV: 0.9787908603913852
FPR: 0.13764207281833943
FDR: 0.08609471020604892
FNR: 0.012627741977478357
ACC: 0.9369586327442222
F1_score: 0.9492193610963362
MCC: 0.8709482562288337
informedness: 0.8497301852041823
markedness: 0.8926961501853361
prevalence: 0.5967372305302
LRP: 7.173477104821427
LRN: 0.014643272334433183
DOR: 489.8821070173794
FOR: 0.021209139608614848
RMSE=  0.2510803999833078


In [15]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
clf4 = KNeighborsClassifier(n_neighbors =12)
clf4.fit(X_train, y_train) 
y_pred4 = clf4.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred4)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred4))
print("RMSE= ",RMSE)

population: 25745
P: 15363
N: 10382
PositiveTest: 15388
NegativeTest: 10357
TP: 15244
TN: 10238
FP: 144
FN: 119
TPR: 0.9922541170344333
TNR: 0.9861298401078791
PPV: 0.9906420587470757
NPV: 0.9885101863473978
FPR: 0.01387015989212098
FDR: 0.009357941252924357
FNR: 0.007745882965566621
ACC: 0.9897844241600311
F1_score: 0.9914474326038177
MCC: 0.9787680257345509
informedness: 0.9783839571423125
markedness: 0.9791522450944736
prevalence: 0.5967372305302
LRP: 71.53876557674643
LRN: 0.007854830723628898
DOR: 9107.613912231558
FOR: 0.011489813652602104
RMSE=  0.10107213186615253


In [16]:
# meta classifier LogisticRegression
lr = LogisticRegression(solver='lbfgs')

# stack esemble method 
from mlxtend.classifier import StackingClassifier
sclf = StackingClassifier(classifiers=[clf1, clf2,clf4], 
                          meta_classifier=lr)

In [17]:
# train and test the model
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE= ",RMSE)

population: 25745
P: 15363
N: 10382
PositiveTest: 15340
NegativeTest: 10405
TP: 15233
TN: 10275
FP: 107
FN: 130
TPR: 0.9915381110460196
TNR: 0.9896937006357157
PPV: 0.9930247718383312
NPV: 0.9875060067275349
FPR: 0.010306299364284338
FDR: 0.00697522816166884
FNR: 0.008461888953980342
ACC: 0.9907943289959216
F1_score: 0.9922808846041103
MCC: 0.9808812324954981
informedness: 0.9812318116817353
markedness: 0.9805307785658659
prevalence: 0.5967372305302
LRP: 96.20699690541846
LRN: 0.008550007894912302
DOR: 11252.269949676493
FOR: 0.012493993272465162
RMSE=  0.09594618806434398
